In [1]:
#This notebook is used to visualize the trend functions
import sys
sys.path.append("..")

from core.common import VBTfunc
import vectorbtpro as vbt
import talib
import numpy as np
from numba import njit
import math
import inspect

from core import constants, strat
from core.indicators import VBTKAMA, VBTBBANDSTREND, VBTMACDBBTREND

from trading_bot.settings import _settings

In [2]:
#modify it depending on what you want to observe
def trend_mode(trend, mode_to_vis ):
    entries=(trend[:]>=mode_to_vis)
    exits=~entries
      
    return entries, exits

VBTTRENDMODE= vbt.IF(
      class_name='VBTPatternEnt',
      short_name='pattern_ent',
      input_names=['macro_trend'],
      param_names=['mode_to_vis'],
      output_names=['entries', 'exits']
 ).with_apply_func(
      trend_mode, 
      takes_1d=True,  
 ) 

In [3]:
class Vis(VBTfunc):
    def __init__(self,input_ust, selector):
        self.close=input_ust.close
        self.trend={}
        if selector=="bbands":
            self.trend=VBTBBANDSTREND.run(input_ust.close).trend
        else:
            self.trend=VBTMACDBBTREND.run(input_ust.close).trend
            
    def trend_mode(self,mode_to_vis):
        t=VBTTRENDMODE.run(self.trend,mode_to_vis=mode_to_vis)
        self.ents=t.entries
        self.exs=t.exits
            
    def manual_calculate_pf(self,mode_to_vis):
        self.trend_mode(mode_to_vis)
        pf=vbt.Portfolio.from_signals(self.close, self.ents,self.exs,freq="1d")
        return pf
        

In [4]:
period="2007_2022_08"
symbol_index="CAC40"
ust=strat.StratHold(period,symbol_index=symbol_index) #it_is_index

  0%|          | 0/39 [00:00<?, ?it/s]

In [5]:
o=Vis(ust,"bbands")

In [6]:
pf=o.manual_calculate_pf(10)

In [7]:
#pf.plot()
pf.plot(column=pf.wrapper.columns[3])

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'mode': 'lines',
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f76a02b1-b0d3-408d-a8b4-ea81ef537412',
              'x': array([datetime.datetime(2007, 1, 2, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2007, 1, 3, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2007, 1, 4, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2022, 8, 29, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 8, 30, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 8, 31, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([37.9964447 , 38.47508621, 36.59735107, ..., 20.64999962,

In [11]:
o.trend["CAC40"]["AC"].vbt.plot()

FigureWidget({
    'data': [{'name': 'AC',
              'showlegend': True,
              'type': 'scatter',
…